In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')
!pwd

Mounted at /content/drive
/content


In [5]:
PATIENT_PATH = "/content/drive/MyDrive/dataset/patient/"
HEALTHY_PATH = "/content/drive/MyDrive/dataset/healthy/"
patient_data = [ PATIENT_PATH + f"patient_{i}.csv" for i in range(0,35)]
healthy_data = [ HEALTHY_PATH + f"healthy{i}.csv" for i in range(0,35)]
patient_df = [pd.read_csv(f"{i}") for i in patient_data]
healthy_df = [pd.read_csv(f"{i}") for i in healthy_data]

In [153]:
for i in range(35):
  patient_df[i].pop('Unnamed: 0')
  healthy_df[i].pop('Unnamed: 0')

In [2]:
def BandPassFilter(df):
  fs=7680
  low_cut=0.1
  high_cut=64.0
  nqy=0.5*fs
  low=low_cut/nqy
  high=high_cut/nqy
  order=2
  pass_data = pd.DataFrame()
  for index in list(df.index):
    data = df.loc[index]
    b,a= scipy.signal.butter(order,[low,high],'bandpass',analog=False)
    y=scipy.signal.filtfilt(b,a,data,axis=0)
    ser = pd.Series(y,name=index)
    pass_data = pass_data.append(ser)
  return pass_data

In [ ]:
''' Segmentation '''

In [155]:
def FFT(df):
    fs = 128 # Sampling rate (128 Hz)
    band_data = pd.DataFrame() 
    for index in list(df.index):
        data = df.loc[index]  # 2 sec of data b/w 0.0-100.0

        # Get real amplitudes of FFT (only in postive frequencies)
        fft_vals = np.absolute(np.fft.rfft(data))
        # Get frequencies for amplitudes in Hz
        fft_freq = np.fft.rfftfreq(len(data), 1.0/fs)
        # Define EEG bands
        eeg_bands = {'Delta': (0, 4),
                     'Theta': (4, 8),
                     'Alpha': (8, 12),
                     'Beta': (12, 30),
                     'Gamma': (30, 45)}

        # Take the mean of the fft amplitude for each EEG band
        eeg_band_fft = dict()
        for band in eeg_bands:  
            freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & (fft_freq <= eeg_bands[band][1]))[0]
            eeg_band_fft[band] = np.mean(fft_vals[freq_ix])

        ser = pd.Series(eeg_band_fft,name=index)
        band_data = band_data.append(ser)
    return band_data

In [163]:
patient_fft = [FFT(BandPassFilter(i)) for i in patient_df]
healthy_fft = [FFT(BandPassFilter(j)) for j in healthy_df]

In [181]:
feature_names = ["Alpha","Beta","Delta","Gamma","Theta"]
patient = [] 
healthy = []
for i in range(0,30):
  patient.append(np.array(patient_fft[i][feature_names]))
  healthy.append(np.array(healthy_fft[i][feature_names]))

(5, 16)